In [1]:
from requests import post
from requests.exceptions import JSONDecodeError
import uuid
import os
from dotenv import load_dotenv

# Для ipynb явно загружаются переменные окружения
load_dotenv('.env')

GIGACHAT_AUTH_URL = os.environ.get("GIGACHAT_AUTH_URL")
GIGACHAT_AUTHORIZE_KEY = os.environ.get("GIGACHAT_AUTHORIZE_KEY")
GIGACHAT_REQUEST_URL = os.environ.get("GIGACHAT_REQUEST_URL")
GIGACHAT_MODEL = os.environ.get("GIGACHAT_MODEL")
MESSAGE_HISTORY_LIMIT = 10

if not GIGACHAT_REQUEST_URL:
    raise ValueError("GIGACHAT_REQUEST_URL не установлена в окружении")

if not GIGACHAT_MODEL:
    raise ValueError("GIGACHAT_MODEL не установлена в окружении")

if not GIGACHAT_AUTH_URL:
    raise ValueError("GIGACHAT_AUTH_URL не установлен в окружении")

if not GIGACHAT_AUTHORIZE_KEY:
    raise ValueError("GIGACHAT_AUTHORIZE_KEY не установлен в окружении")

# Обновлять токен доступа, если до окончания действия осталось 60 секунд
REFRESH_ACCESS_TOKEN_BEFORE_S = 60
__access_token = ""
__access_token_expires = ""

def refresh_access_token():
    """Refresh access_token for llm and set access_token_expires time

    Returns:
        Boolean: 
            True if refresh is success
            False if token is not refreshed
    """

    global __access_token
    global __access_token_expires

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json",
        "RqUID": f"{uuid.uuid4()}",
        "Authorization": f"Basic {GIGACHAT_AUTHORIZE_KEY}"
    }

    payload = {
        "scope": "GIGACHAT_API_PERS"
    }

    try:
        response = post(
            url=GIGACHAT_AUTH_URL,
            headers=headers,
            data=payload
        )

        response_json = response.json()

        __access_token = response_json.get("access_token")
        __access_token_expires = response_json.get("expires_at")

        if __access_token is None or __access_token_expires is None:
            print("Ошибка при выполнении запроса на обновление токена доступа",
                  "\nВ полученных данных отсутствуют 'access_token' или 'expires_at'",
                  f"\nОтвет сервиса: {response_json}")
            return False
    except JSONDecodeError as ex:
        print(f"Ошибка при парсинге ответа от сервиса при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    except Exception as ex:
        print(f"Ошибка при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    
    return True


In [2]:
is_refresh_success = refresh_access_token()
print(is_refresh_success)

True


In [3]:
from datetime import datetime

# При обращении к функции передачи сообщения нужна проверка действительности токена и его обновление

def refresh_access_token_if_expire(func):
    def wrapper(*args, **kwargs):
        # Проверка действительности токена
        # Преобразование в секунды для корректной работы с datetime
        token_expire_time_s = __access_token_expires / 1000.0
        token_expire_datetime = datetime.fromtimestamp(token_expire_time_s)
        # Остаток времени до истечения токена в секундах
        time_left_s = (token_expire_datetime - datetime.now()).total_seconds()

        print(
                "Refreshing access token",
                f"Expire datetime is: {token_expire_datetime}",
                f"Seconds left: {time_left_s}"
            )
        
        # Если действие токена скоро закончится или уже закончилось - обновить
        if time_left_s < REFRESH_ACCESS_TOKEN_BEFORE_S:
            print(
                "Refreshing access token",
                f"Expire datetime is: {token_expire_datetime}",
                f"Seconds left: {time_left_s}"
            )
            refresh_access_token()
        
        return func(*args, **kwargs)

    return wrapper

In [ ]:
# Нужна выгрузка истории сообщений по достижении определенного лимита, чтобы не хранить всё в оперативной памяти
# Нужно лимитировать историю сообщений, передаваемую LLM

# Формат хранения истории сообщений
# message_history = [
#     {
#         "role": "user",
#         "content": "request text"
#     }
# ]

import json

message_history = []
message_history_ids = []

In [5]:
# bot_make_request("Привет! Какие у тебя возможности?")

In [6]:
# print(message_history)
# print(message_history_ids)

In [7]:
# bot_make_request("Составь план действий. Хочу написать скрипт, который будет отправлять в группу в Telegram информацию о загруженности ВМ (cpu, ram, дисковое пространство). Хочу использовать python-telegram-bot")

In [8]:
# print(message_history)
# print(message_history_ids)

In [9]:
from pathlib import Path

__username = None
__save_message_history = False

def bot_load_user_config(config_path="conf/user_settings.json"):
    global __username
    global __save_message_history

    path = Path(config_path)

    if (path.exists()):
        try:
            with open(path, "r", encoding="utf-8") as settings_file:
                settings = json.load(settings_file)

                __username = settings.get("username")
                __save_message_history = settings.get("save_message_history", False)

                if __username is None:
                    print(f"[ERROR] Не удается найти имя пользователя в файле конфигурации {config_path} (username)")
                    return False
                
                if __save_message_history is None:
                    print(f"[WARN] Флаг сохранения истории сообщений установлен в False, так как не был найден в файле конфигурации {config_path} (save_message_history)")
                
                return True
        except JSONDecodeError as ex:
            print("[ERROR] Ошибка при парсинге файла конфигурации в JSON", ex)
        except Exception as ex:
            print("[ERROR] Непредвиденная ошибка при чтении файла конфигурации", ex)
    else:
        print(f"[ERROR] Не удается найти файл конфигурации пользователя по пути {path}")
        return False

In [10]:
bot_load_user_config()

True

In [11]:
print(__username, __save_message_history)

host True


In [12]:
from enum import Enum

class FileFormat(Enum):
    JSON = "json"
    TEXT = "txt"

# Функция для сохранения файла
def utils_save_file(path: Path, format: str, data) -> bool:
    print(path)
    if not path.exists():
        print("[WARN] Путь к файлу не существует. Попытка создать файл")
        try:
            path.touch()
        except Exception as ex:
            print("[ERROR] Ошибка при создании файла", ex)
            return False
    

    if format == FileFormat.JSON:
        try:
            with path.open("w", encoding="utf-8") as file:
                json.dump(data, file, indent=4, ensure_ascii=False)
        except Exception as ex:
            print("[ERROR] Ошибка при сохранении JSON файла")
            return False
    elif format == FileFormat.TEXT:
        try:
            with path.open("w", encoding="utf-8") as file:
                file.write(str(data))
        except Exception as ex:
            print("[ERROR] Ошибка при сохранении текстового файла")
            return False
    else:
        print("[ERROR] Неизвестный формат файла. Сохранение не будет выполнено")
    
    return True

# Функция для сохранения истории сообщений
def bot_save_message_history(message_history: list = None, message_history_ids: list = None) -> bool:
    try:
        if message_history is None:
            print("[ERROR] Не задана история сообщений для сохранения")
            return False
        elif message_history_ids is None:
            print("[ERROR] Не задан список идентификаторов для сопоставления истории сообщений с сохранением")
            return False
        elif len(message_history) == 0:
            print("[WARN] История сообщений пуста. Сохранение не будет выполнено")
            return False
        elif __username is None:
            print("[ERROR] Пользователь для сохранения истории сообщений не известен")
            return False
    except Exception as ex:
        print("[ERROR] Ошибка при сохранении истории сообщений", ex)

    message_history_object = {}

    path = Path(f"data/history/{__username}.json")

    if (path.exists()):
        # Если уже есть сохраненные сообщения, нужно вычитать их в память и выполнить дозапись
        with path.open("r", encoding="utf-8") as file:
            message_history_object = json.load(file)
            
            if "message_history_ids" not in message_history_object:
                print("[ERROR] В файле с сохраненной историей сообщений некорректная структура. Отсутствует поле 'message_history_ids'")
                return False
            
            if "message_history" not in message_history_object:
                print("[ERROR] В файле с сохраненной историей сообщений некорректная структура. Отсутствует поле 'message_history'")
                return False

            for index, id in enumerate(message_history_ids):
                
                # Пропустить добавление элементов, которые уже есть в файле
                if id in message_history_object.get("message_history_ids"):
                    continue
                
                # Добавить элементы истории сообщений на запись
                message_history_object["message_history_ids"].append(id)
                message_history_object["message_history"].append(message_history[index])
    else:
        # Если файла с данными нет, все элементы истории сообщений пойдут в запись
        message_history_object["message_history_ids"] = message_history_ids
        message_history_object["message_history"] = message_history

    utils_save_file(path, FileFormat.JSON, message_history_object)

    return True

In [13]:
# bot_save_message_history(message_history=message_history, message_history_ids=message_history_ids)

In [14]:
import copy

@refresh_access_token_if_expire
def bot_make_system_request(request_message: str, message_context: list = None) -> dict:
    """Make request to llm without user message history editing

    Args:
        request_message (str): request to llm
        message_history (list, optional): previous messages to|from llm. Defaults to None. If need to use the messages for context

    Returns:
        dict: llm answer in format: {
            "role": "assistant",
            "message": assistant_message
        }

        None if something went wrong
    """
    if message_context is not None:
        request_messages = copy.deepcopy(message_context)
    else:
        request_messages = []

    # Т.к. системное сообщение может быть направлено только первым, запросы программы (не пользователя) также передаются под ролью user
    request_messages.append({
        "role": "user",
        "content": request_message
    })

    print(request_messages)

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {__access_token}"
    }

    body = json.dumps({
        "model": GIGACHAT_MODEL,
        "messages": request_messages
    })

    try:
        response = post(
            url=GIGACHAT_REQUEST_URL,
            headers=headers,
            data=body
        )

        response_json = response.json()

        assistant_message = response_json.get("choices")[0].get("message").get("content")

        if assistant_message is None:
            print(f"[ERROR] В полученных данных отсутствует сообщение модели")
            return None
        
        # Вернуть ответ модели
        return {
            "role": "assistant",
            "message": assistant_message
        }
    except JSONDecodeError as ex:
        print(f"[ERROR] Ошибка при парсинге ответа от сервиса при выполнении запроса на обновление токена доступа\n{ex}")
        return None
    except Exception as ex:
        print("[ERROR] Ошибка при выполнении запроса к LLM", ex)
        return None

In [15]:
# TODO

__context = None
__context_prompt = None

# Функция для генерации контекста на основе предыдущих сообщений
def bot_generate_context(message_history: list = None):
    global __context

    if __context_prompt is None:
        print("[ERROR] Промпт для генерации контекста не задан")
        return False

    response_message = bot_make_system_request(__context_prompt, message_context=message_history)

    if response_message is not None:
        __context = response_message.get("message")
    else:
        return False

    return True



# Функция для сохранения контекста (под капотом использовать функцию сохранения файла)
def bot_save_context(context_data: str = None):
    path = Path(f"data/context/{__username}")

    if context_data is not None:
        utils_save_file(path=path, format=FileFormat.TEXT, data=context_data)
    else:
        print("[WARN] Контекст пуст, сохранение не будет произведено")

def bot_load_context_prompt(context_path="conf/app/context_prompt"):
    global __context_prompt

    path = Path(context_path)

    if path.exists():
        with path.open("r", encoding="utf-8") as file:
            data = file.read()

            if data:
                __context_prompt = data
                return True
            else:
                print("[ERROR] Файл промпта для генерации контекста пуст. Промпт генерации контекста не задан")
                return False
    else:
        print(f"[ERROR] Указанный путь к файлу контекста не существует ({path})")
        return False

In [16]:
bot_load_context_prompt()

True

In [17]:
print(__context_prompt)

Сгенерируй контекст на основе истории сообщений
Контекст должен включать сведения о пользователе, если они присутствуют в истории сообщений
Также в контексте необходимо зафиксировать предмет разговора - о чем ведется речь в истории сообщений
Если в истории сообщений есть информация о конкретных предметах, технологиях и т.д., необходимо зафиксировать это в контексте

Пример контекста:
Пользователь Василий интересуется разработкой на Python. Он задает вопросы о том как сохранять файлы в Python, выполнять запросы к API.
Также Василий планирует применить полученные знания в рабочих задачах. Он хочет создать скрипт на Python, который будет присылать уведомления в Telegram
о нехватке ресурсов на хосте на базе ОС Linux. Можно подсказывать ему о технологиях, которые позволят ему реализовать свою идею.

Сделай контекст как можно меньше, с сохранением смысла

Не используй это сообщение в качестве источника информации при генерации контекста.
Не включай в контекст информацию в которой не уверен.


In [18]:
# bot_generate_context(message_history=message_history)

In [19]:
# print(__context)

In [20]:
# bot_save_context(__context)

In [21]:
@refresh_access_token_if_expire
def bot_make_request(request_message: str):
    # Действия при превышении объема истории сообщений заданного лимита
    if len(message_history) >= MESSAGE_HISTORY_LIMIT:
        # Сохранить сообщения по достижении лимита
        bot_save_message_history(message_history, message_history_ids)
        # Сгенерировать контекст - он будет подставляться вместо истории сообщений при отправке к сервису LLM
        bot_generate_context(message_history)

        # Очистить предыдущую историю в памяти программы
        message_history.clear()
        message_history_ids.clear()

    message_history.append({
        "role": "user",
        "content": request_message
    })

    # Идентификаторы сообщений хранятся в отдельной структуре
    # согласовываются по индексу
    # message_history[0] -> message_history_ids[0]
    message_history_ids.append(str(uuid.uuid4()))

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {__access_token}"
    }

    request_messages = []

    if __context is None:
        request_messages = message_history
    else:
        request_messages = [{
            "role": "user",
            "content": f"Используй этот контекст истории общения для более точных ответов: {__context}"
        }].extend(message_history)

    body = json.dumps({
        "model": GIGACHAT_MODEL,
        "messages": request_messages
    })

    try:
        response = post(
            url=GIGACHAT_REQUEST_URL,
            headers=headers,
            data=body
        )

        response_json = response.json()

        assistant_message = response_json.get("choices")[0].get("message").get("content")

        if assistant_message is None:
            print(f"[ERROR] В полученных данных отсутствует сообщение модели")
            return False
        
        # Добавить ответ модели в историю сообщений
        message_history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        # Сохранение идентификатора сообщения
        message_history_ids.append(str(uuid.uuid4()))
    except JSONDecodeError as ex:
        print(f"[ERROR] Ошибка при парсинге ответа от сервиса при выполнении запроса на обновление токена доступа\n{ex}")
        return False
    except Exception as ex:
        print("[ERROR] Ошибка при выполнении запроса к LLM", ex)
        return False
    
    return True

In [22]:
from rich.console import Console
from rich.markdown import Markdown
from rich.theme import Theme

def bot_print_markdown_message(message: str = ""):
    if len(message) == 0:
        print("[WARN] Нет данных для отображения")
    
    console = Console(theme=Theme({
        "markdown.h1": "bold bright_magenta",
        "markdown.h2": "bold bright_cyan",
        "markdown.h3": "bold bright_green",
        "markdown.h4": "bold bright_yellow",
        "markdown.h5": "bold bright_blue",
        "markdown.h6": "bold bright_red",
        "markdown.strong": "bold bright_white",
        "markdown.emphasis": "italic bright_cyan",
        "markdown.strike": "strike bright_black",
        "markdown.code": "bright_green on grey11",
        "markdown.code_block": "bright_yellow on grey0",
        "markdown.link": "underline bright_blue",
        "markdown.link_url": "italic bright_cyan",
        "markdown.block_quote": "dim bright_magenta",
    }))

    markdown_text = Markdown(message)

    console.print(markdown_text)

In [23]:
# bot_print_markdown_message(message_history[-1].get("content"))

In [24]:
def bot_print_message_history(message_history: list = None, show_last: int = 10):
    """Show 'show_last' messages of message history

    Args:
        message_history (list, optional): list of messages with content. Defaults to None.
        show_last (int, optional): sets how many messages to display. Defaults to 10.
    """
    if message_history is None or len(message_history) == 0:
        print("[WARN] История сообщений пуста")
    
    for message_item in message_history[-show_last:]:
        if message_item.get("role") == "user":
            print(f"\033[1m\033[34m======{__username if __username is not None else "Пользователь"}======\033[0m")
        
        if message_item.get("role") == "assistant":
            print("\033[1m\033[92m======Model======\033[0m")

        if "content" in message_item:
            bot_print_markdown_message(message_item.get("content"))

In [25]:
bot_print_message_history(message_history)

[WARN] История сообщений пуста


In [28]:
DISPLAY_MESSAGE_HISTORY_COUNT = 6

def show_message_history():
    # Создание дубликата истории сообщений, т.к. возможна мутация
    messages_to_display = copy.deepcopy(message_history)
    messages_to_display_count = len(messages_to_display)

    # Если в памяти мало сообщений для отображения
    if messages_to_display_count < DISPLAY_MESSAGE_HISTORY_COUNT:
        path_to_message_history_file = Path(f"data/history/{__username}.json")
        
        is_history_file_exists = path_to_message_history_file.exists()
        # История пуста, файл с историей отсутствует
        if messages_to_display_count == 0:
            if not is_history_file_exists:
                print("[WARN] История сообщений пуста")
                return
        
        if is_history_file_exists:
            # Догрузить сообщения из файла с сохранениями
            with path_to_message_history_file.open("r", encoding="utf-8") as file:
                message_history_object = json.load(file)

                if "message_history_ids" not in message_history_object:
                    print("[ERROR] В файле с сохраненной историей сообщений некорректная структура. Отсутствует поле 'message_history_ids'")
                    return
            
                if "message_history" not in message_history_object:
                    print("[ERROR] В файле с сохраненной историей сообщений некорректная структура. Отсутствует поле 'message_history'")
                    return
                
                file_message_histody_len = len(message_history_object["message_history"])
                # Обход списка истории сообщений в обратном порядке
                for shift, message_id in enumerate(reversed(message_history_object["message_history_ids"])):
                    print(shift, message_id)
                    # Если message_id сообщения уже есть в памяти, загрузка такого сообщения не требуется
                    if message_id in message_history_ids:
                        continue
                    
                    # Вставка сообщений в начало истории сообщений в памяти (сообщения в файле старее, чем сообщения в памяти)
                    messages_to_display.insert(0, message_history_object["message_history"][file_message_histody_len - 1 - shift])

                    # При доборе сообщений до нужного количества, завершить обход
                    if len(messages_to_display) >= DISPLAY_MESSAGE_HISTORY_COUNT:
                        break
        elif messages_to_display_count == 0:
            print("[WARN] История сообщений пуста")

    bot_print_message_history(messages_to_display, 6)

In [29]:
show_message_history()

0 358dfd92-1a68-49de-a4ad-daa2ce87d311
1 d7561b9f-67fb-4c4e-849f-8ae7548f7090
2 7245f340-2195-4293-a642-6312e390faea
3 017ef57f-a5e7-4c29-8a5e-3a8a3c5386e8
======host======


Привет! Какие у тебя возможности?

======Model======


Вот основные мои возможности:                                                                                      

📌 Основные функции:                                                                                               

 • Работа с текстом: помогу писать тексты любого формата (от эссе до сложных технических статей), отвечать на      
   вопросы, помогать с учебой, решать задачи и упражнения разного уровня сложности.                                
 • Разбор текста: детально проанализирую любой документ, статью, отрывок — найду ошибки, исправлю стилистические   
   погрешности, сделаю текст грамотнее и понятнее.                                                                 
 • Помогу разобраться с программированием: напишу код, отлажу программу, дам советы по алгоритмам и структурам     
   данных.                                                                                                         
 • Решение задач: математика, логика, химия, физика, биология — разберём любую задачу пошагово.                    

🔗 Дополнительно:                                                                                                  

 • Можно обсудить любую тему: философия, история, культура, спорт, искусство, технологии, политика и многое другое.
 • Помогу выбрать правильное решение в разных жизненных ситуациях.                                                 

🎨 🖼️ Возможности вне стандартного функционала:                                                                    

 • Могу создать небольшие изображения или иллюстрации, используя простой текстовый интерфейс.                      
 • Иногда предлагаю интересные идеи и подсказки, например, придумать необычные сюжеты, истории или сценарии.       

Что именно интересует — пиши!

======host======


Составь план действий. Хочу написать скрипт, который будет отправлять в группу в Telegram информацию о             
загруженности ВМ (cpu, ram, дисковое пространство). Хочу использовать python-telegram-bot

======Model======


✅ План действий по созданию скрипта мониторинга нагрузки виртуальных машин и отправки уведомлений через Telegram  

-------------------------------------------------------------------------------------------------------------------

1. Подготовка окружения и установка зависимостей                                                                   

Что нужно подготовить:                                                                                             

 • Python 3.x версии.                                                                                              
 • Установленный пакет python-telegram-bot (pip install python-telegram-bot).                                      
 • Доступ к API токену Telegram для работы с ботом.                                                                
 • Сервер или VM с возможностью подключения SSH и установки необходимых утилит.                                    

Установка зависимостей:                                                                                            

                                                                                                                   
 pip install python-telegram-bot psutil                                                                            
                                                                                                                   

-------------------------------------------------------------------------------------------------------------------

2. Получение токена Telegram бота                                                                                  

Создай нового бота в Telegram через @BotFather:                                                                    

 • Запусти команду /newbot.                                                                                        
 • Следуй инструкциям, чтобы получить токен вашего бота.                                                           

-------------------------------------------------------------------------------------------------------------------

3. Мониторинг загрузки CPU, RAM и диска с помощью библиотеки psutil                                                

Библиотека psutil поможет собирать данные о загрузке ресурсов машины:                                              

                                                                                                                   
 import psutil                                                                                                     
 from datetime import datetime                                                                                     
                                                                                                                   
 def get_load_data():                                                                                              
     cpu_percent = psutil.cpu_percent(interval=1)                                                                  
     memory = psutil.virtual_memory()                                                                              
     disk_usage = psutil.disk_usage('/')                                                                           
                                                                                                                   
     return {                                                                                                      
         'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M'),                                                   
         'cpu_percent': f"{cpu_percent:.2f}%",                                                                     
         'ram_total': f"{memory.total / (1024 * 1024):.2f} MB",                                                    
         'ram_used': f"{memory.used / (1024 * 1024):.2f} MB",                                                      
         'disk_total': f"{disk_usage.total 